# Fine tune Qwen 2 using torchrun

Note: GPU instance types are required to execute this notebook and has been tested with ml.g5.xlarge 

## Install dependencies

In [ ]:
!micromamba install -y --freeze-installed  "nvidia::cuda>=11.8,<11.9"
!nvcc --version
!ls /usr/local/cuda/lib64/libcudart.so.11.8.89
!sudo rm -f /usr/lib/libcudart.so
!sudo ln -s /usr/local/cuda/lib64/libcudart.so.11.8.89 /usr/lib/libcudart.so
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory && pip install -e ".[torch,metrics]"
!llamafactory-cli version

## Train model

In [ ]:
!llamafactory-cli train qwen2-0.5-qlora-sft.yaml

In [4]:
!pip install peft deepspeed optimum accelerate bitsandbytes --upgrade --quiet

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Load model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
model_name_or_path = "./saves/qwen2-0.5B/qlora/sft/"
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response